In [1]:
from clean_tabular import CleanData, CleanImages, MergedData
import os
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split, Dataset, SubsetRandomSampler
import torchvision.transforms as transforms
import re
import multiprocessing
import torchvision
from torchbearer import Trial
from torch.utils.tensorboard import SummaryWriter
from torch.optim import lr_scheduler
from torchbearer.callbacks import TensorBoard
from torch.nn import Module
import matplotlib.pyplot as plt
import seaborn as sns
from torch import nn
import torch.optim as optim
from pytorch_scratch_classification import Dataset
from torchvision import models, datasets
import copy
import time
import pandas as pd
import numpy as np

from collections import defaultdict, Counter

[nltk_data] Downloading package omw-1.4 to /Users/aj8/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/Users/aj8/opt/anaconda3/envs/scaper_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

merged = MergedData()
encoder_dict, decoder_dict = merged.vocab_encoder()
word_df = merged.feature_target_tuple(context_size=2).loc[:, ['context_encoded', 'context', 'target', 'target_encoded']]

{0: 'Home & Garden ', 1: 'Baby & Kids Stuff ', 2: 'DIY Tools & Materials ', 3: 'Music, Films, Books & Games ', 4: 'Phones, Mobile Phones & Telecoms ', 5: 'Clothes, Footwear & Accessories ', 6: 'Other Goods ', 7: 'Health & Beauty ', 8: 'Sports, Leisure & Travel ', 9: 'Appliances ', 10: 'Computers & Software ', 11: 'Office Furniture & Equipment ', 12: 'Video Games & Consoles '}
{'Home & Garden ': 0, 'Baby & Kids Stuff ': 1, 'DIY Tools & Materials ': 2, 'Music, Films, Books & Games ': 3, 'Phones, Mobile Phones & Telecoms ': 4, 'Clothes, Footwear & Accessories ': 5, 'Other Goods ': 6, 'Health & Beauty ': 7, 'Sports, Leisure & Travel ': 8, 'Appliances ': 9, 'Computers & Software ': 10, 'Office Furniture & Equipment ': 11, 'Video Games & Consoles ': 12}
{0: 'Home & Garden ', 1: 'Baby & Kids Stuff ', 2: 'DIY Tools & Materials ', 3: 'Music, Films, Books & Games ', 4: 'Phones, Mobile Phones & Telecoms ', 5: 'Clothes, Footwear & Accessories ', 6: 'Other Goods ', 7: 'Health & Beauty ', 8: 'Sports

In [4]:
word_df

,context_encoded,context,target,target_encoded
0,21039,second,hand,15317
0,5638,attempted,hand,15317
0,25310,declare,hand,15317
0,21220,use,hand,15317
1,15317,hand,attempted,5638
...,...,...,...,...
309956,17668,alloy,multi,16996
322927,1843,monitor,dell,25113
322927,24538,optiplex,dell,25113
322927,20501,touchscreen,dell,25113


In [115]:
def embedder(col='product_description'):
    print(merged.merged_frame['product_description'].map(merged.vocab_encoder()[0]))
    print(len(merged.merged_frame['product_description'].map(merged.vocab_encoder()[0])))
    merged.merged_frame['product_description'].map(merged.vocab_encoder()[0])

In [5]:
class WordDataset(Dataset):
    def __init__(self, win_size=3):
        word_cls = MergedData()
        cleaned_product = word_cls.clean_prod()
        self.text = cleaned_product[0]
        self.all_unique_words = cleaned_product[1]
        self.window_size = win_size
        self.vocab_size  = word_cls.get_word_set(col='product_description')[1]
        print(self.vocab_size)
        self.int2wrd = word_cls.vocab_encoder()[1]
        print(self.int2wrd)
        self.wrd2int = word_cls.vocab_encoder()[0]
        print(self.wrd2int)

    def __len__(self):
        return len(self.all_wrds) - self.window_size

    def __getitem__(self, idx):
        X = torch.LongTensor([self.wrd2int[ind] for ind in self.text[idx: idx + self.window_size]])        
        y = self.wrd2int[self.text[idx + self.window_size]]
        return X, y

In [ ]:
class Network(nn.Module):
    def __init__(self, embedding_dim, dense_dim, hidden_dim, n_layers):
        pass

In [4]:
test_sentence = 'the quick brown fox jumped over lazy dog'.split()
CONTEXT_SIZE = 2
ngrams = [
    (
        [test_sentence[i - j - 1] for j in range(CONTEXT_SIZE)],
        test_sentence[i]
    )
    for i in range(CONTEXT_SIZE, len(test_sentence))
]

In [5]:
skip_gram = [
    [
        list(np.array([[test_sentence[i - j], test_sentence[i+j]] for j in range(1, CONTEXT_SIZE+1)]).flatten()),
        test_sentence[i]
    ]
    for i in range(CONTEXT_SIZE, len(test_sentence)-CONTEXT_SIZE)
]

skip_gram

[[['quick', 'fox', 'the', 'jumped'], 'brown'],
 [['brown', 'jumped', 'quick', 'over'], 'fox'],
 [['fox', 'over', 'brown', 'lazy'], 'jumped'],
 [['jumped', 'lazy', 'fox', 'dog'], 'over']]

In [101]:
np.array(skip_gram)[:, 1]

/var/folders/sv/k_x6s66n6ls2xvtk5677t8_r0000gn/T/ipykernel_3663/2460276757.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(skip_gram)[:, 1]


array(['brown', 'fox', 'jumped', 'over'], dtype=object)

In [77]:
skip_gram

AttributeError: 'list' object has no attribute 'to_array'

In [75]:
r1

[[['brown', 'jumped'], ['quick', 'over'], 'fox'],
 [['fox', 'over'], ['brown', 'lazy'], 'jumped'],
 [['jumped', 'lazy'], ['fox', 'dog'], 'over']]

In [49]:
df = pd.DataFrame(columns=['word', 'context'])

In [62]:
l1 = [1,2]
l1 = *'ab'

SyntaxError: can't use starred expression here (2375518790.py, line 2)

In [44]:
print(skip_gram)

[([['brown', 'jumped'], ['quick', 'over']], 'fox'), ([['fox', 'over'], ['brown', 'lazy']], 'jumped')]


In [30]:
first

[([['brown', 'jumped'], ['quick', 'over']],),
 ([['fox', 'over'], ['brown', 'lazy']],),
 ([['jumped', 'lazy'], ['fox', 'dog']],)]